In [8]:
import glob, os
import numpy as np
import pandas as pd
import soundfile as sf
from functools import partial
from pathlib import Path



import yamnet_model.params as yamnet_params
import yamnet_model.yamnet as yamnet_model

from src.features import FeatureExtractor


from omegaconf import OmegaConf

from src.data import load_metadata, find_paths
# from src.features.YAMNetExtractor import YAMNetExtractor
import src.utils

from omegaconf import OmegaConf
conf = OmegaConf.load("./config.yaml")

# Prepare topic task

We need to create audio fragments and yamnet embeddings for the topic task


## Step 1: Create audio files

In [3]:
# TODO DEZE code checken
from tts import tts

# Directory where audio files will be stored
query_audio_dir = os.path.join("../", conf.query_audio_dir)
target_dir = os.path.join("../", conf.dataset_path, 'topic_task')

# Create audio for train set
topics_df_path = os.path.join(target_dir, 'podcasts_2020_topics_train.xml')
create_audiofiles(topics_df_path, target_dir)

# Create audio for test set
topics_df_path = os.path.join(target_dir, 'podcasts_2020_topics_test.xml')
create_audiofiles(topics_df_path, target_dir)

# HIerna omzetten naar .ogg

## Step 2: Create yamnet embeddings

In [9]:

class YAMNetExtractor(FeatureExtractor):
    """Class for feature extraction with YAMNet.

    example:
    ex = YAMNetExtractor()
    ex.embedding(input_paths, output_paths, embed_paths)
    """

    def __init__(self, logfile="./log_yamnet", yamnet_path="./yamnet_class_map.csv"):
        """Init method for YAMNetExtractor."""
        super().__init__(logfile=logfile)
        self.model_checkpoint = os.path.join("./yamnet_model/yamnet.h5")
        
        #oud
        #self.class_names = os.path.join(
        #    os.getenv("YAMNET_PATH"), "yamnet_class_map.csv"
        #)
        
        # new
        print("init with yamnet path: ", yamnet_path)
        self.class_names = yamnet_path
        self.sample_rate = 44100

    def embedding(self, input_paths, output_paths):
        """Extract YAMnet features with opensmile using a single process."""
        save_embedding = True

        paths = list(zip(input_paths, output_paths))

        params = yamnet_params.Params(
            sample_rate=self.sample_rate, patch_hop_seconds=0.48
        )

        class_names = yamnet_model.class_names(self.class_names)
        yamnet = yamnet_model.yamnet_frames_model(params)
        yamnet.load_weights(self.model_checkpoint)

        func = partial(
            self._embed,
            yamnet=yamnet,
            params=params,
            class_names=class_names,
            save_embedding=save_embedding,
        )

        self.single_process(func, paths)

    @staticmethod
    def _embed(paths, yamnet, params, class_names, save_embedding=False):
        """Individual YAMnet extraction process."""

        input_path, embed_path = paths
        #print("[del] paths, should exist: ", input_path)
        input_path_exists, output_path_exists = FeatureExtractor.feature_path_checker(
            input_path, embed_path
        )
        
        print("[del] will it be true: ", input_path_exists and not output_path_exists)
        print('Should exist: ', input_path, input_path_exists)
        print('Should not exist: ', embed_path, output_path_exists)
        if input_path_exists and not output_path_exists:
            print("[del] if statement TRUE! reading ", input_path)
            wav_data, sr = sf.read(input_path, dtype=np.int16)
            print("[del] wav_data: ", wav_data)
            waveform = np.mean(wav_data, axis=1) / 32768.0

            approx_size = int(
                len(waveform) / params.sample_rate / params.patch_hop_seconds
            )  # approximate (overestimated) size of output
            embedding = np.zeros((approx_size, 1024))
            score = np.zeros((approx_size, 521))

            waveform_size = len(waveform)
            i = 0
            n_seconds = 300
            di = int(n_seconds * params.sample_rate)  # 5min segments

            real_size = 0
            while i <= waveform_size:
                scores, embeddings, spectrogram = yamnet(
                    waveform[i : i + di + int(0.47 * params.sample_rate)]
                ) # the 0.47s has to do with yamnet windowing
                scores = scores.numpy()
                print("[del] scores: ", scores)
                embeddings = embeddings.numpy()

                embedding[real_size : real_size + len(scores)] = embeddings
                score[real_size : real_size + len(scores)] = scores

                real_size += len(scores)
                i += di

            if save_embedding:
                print("[del] Will also save the embedding, to: ", embed_path)
                _, _ = FeatureExtractor.feature_path_checker(
                    input_path, embed_path
                )  # also create embed path if necessary

                df = pd.DataFrame(embedding)
                df["time (s)"] = np.arange(len(embedding)) * 0.48
                df.set_index("time (s)", inplace=True)
                df.astype(np.float16).to_hdf(
                    embed_path, "embedding", mode="w", complevel=6
                )
                del df

#             df = pd.DataFrame(score, columns=class_names)
#             df["time (s)"] = np.arange(len(score)) * 0.48
#             df.set_index("time (s)", inplace=True)
#             df.astype(np.float16).to_hdf(output_path, "score", mode="w", complevel=6)

            del df
            del embedding
            del score
            del spectrogram



In [19]:


def find_files(directory, ext='.ogg'):
    file_list = []
    for path in Path(directory).rglob('*{}'.format(ext)):

        file_list.append(str(path))
        
    return file_list


def get_output_paths(file_list, output_folder, ext=".h5"):
    
    
    outputs = []
    for f in file_list:
        spl = os.path.normpath(f).split(os.path.sep)
        new_lastname = spl[-1].split('.')[0] + ext

        filepath = os.path.join(output_folder, spl[-3], spl[-2], new_lastname)
        outputs.append(filepath)
    return outputs
        

In [24]:
# Load the configuration
# conf = OmegaConf.load("./config.yaml")
query_audio_dir = conf.query_audio_dir
# query_audio_dir = 'test'
print(query_audio_dir)

# Where the yamnet embeddings will be stored
output_folder = conf.query_embed_dir
print(output_folder)

# Find paths that need to be converted
# input_paths = find_paths(subset, input_path, ".ogg")
input_paths = find_files(query_audio_dir, ext='.ogg')
# input_paths = input_paths[-2:]
print(input_paths)
embed_paths = get_output_paths(input_paths, output_folder, ext=".h5")
# embed_paths = embed_paths[-2:]
print(embed_paths)

../data/sp/podcasts-no-audio-13GB/topic_task/query_audio
../data/sp/yamnet/query_embedding
['../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/8.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/2.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/3.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/1.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/4.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/5.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/7.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/6.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/12.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/13.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/39.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/

In [25]:
THE_YAMNET_PATH = './yamnet_model/yamnet_class_map.csv'

# Run the YAMnet feature extraction
ex = YAMNetExtractor(logfile=os.path.join(conf.features_output_path, "log_yamnet"), yamnet_path = THE_YAMNET_PATH)
ex.embedding(input_paths, embed_paths)

init with yamnet path:  ./yamnet_model/yamnet_class_map.csv


 14%|█▎        | 9/66 [00:00<00:01, 44.90it/s]

[del] will it be true:  False
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/8.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/slow/8.h5 True
[del] will it be true:  False
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/2.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/slow/2.h5 True
[del] will it be true:  False
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/3.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/slow/3.h5 True
[del] will it be true:  False
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/1.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/slow/1.h5 True
[del] will it be true:  False
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/4.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/slow/4.h5 True
[del] will

 17%|█▋        | 11/66 [00:00<00:03, 15.23it/s]

[del] scores:  [[9.9861240e-01 5.6598088e-07 2.5713882e-05 ... 1.5849039e-11
  4.9613419e-07 5.7956717e-21]
 [9.9984169e-01 3.8143093e-09 2.3618340e-04 ... 1.2355956e-12
  1.3798845e-07 3.8738124e-25]
 [9.9952209e-01 1.5191336e-08 6.1431527e-04 ... 1.8101767e-10
  1.3848341e-06 2.4287634e-23]
 [9.9281478e-01 3.6153513e-05 4.2456388e-03 ... 6.5273169e-05
  5.3465366e-04 1.0045701e-09]]
[del] Will also save the embedding, to:  ../data/sp/yamnet/query_embedding/query/fast/39.h5
[del] will it be true:  True
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/11.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/fast/11.h5 False
[del] if statement TRUE! reading  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/11.ogg
[del] wav_data:  [[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
[del] scores:  [[9.9887967e-01 9.4522948e-06 2.4582148e-03 ... 1.0036667e-06
  3.0915096e-05 2.3642477e-12]
 [9.9998665e-01 6.0352272e-18 3.031212

...

[del] wav_data:  [[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
[del] scores:  [[9.9585009e-01 6.1002479e-06 1.3538003e-03 ... 1.6383980e-06
  3.5923719e-04 1.0943265e-12]
 [9.9967140e-01 1.1860781e-08 6.5842271e-04 ... 2.7326155e-10
  1.2391592e-05 5.0234549e-20]
 [9.9087346e-01 4.8243335e-05 1.0725051e-02 ... 4.3313229e-09
  6.5243821e-06 2.0316589e-19]
 [9.9782681e-01 2.9927492e-04 2.2152185e-02 ... 2.1681496e-06
  9.5348063e-05 3.6636555e-16]
 [9.5855147e-01 1.3949573e-03 9.8190904e-03 ... 7.5682998e-04
  8.8056922e-04 7.6867686e-09]]
[del] Will also save the embedding, to:  ../data/sp/yamnet/query_embedding/query/fast/34.h5
[del] will it be true:  True
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/20.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/fast/20.h5 False
[del] if statement TRUE! reading  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/20.ogg
[del] wav_data:  [[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0

 94%|█████████▍| 62/66 [00:08<00:00,  5.42it/s]

[del] scores:  [[9.9969870e-01 7.3945102e-08 2.1308661e-03 ... 1.9350127e-12
  2.3663684e-08 7.8611857e-26]
 [9.9993038e-01 1.7640061e-14 1.5585084e-06 ... 2.0266099e-21
  2.3794027e-12 0.0000000e+00]
 [9.9995697e-01 2.2968360e-13 2.9549061e-05 ... 2.5972502e-21
  3.3091745e-12 0.0000000e+00]]
[del] Will also save the embedding, to:  ../data/sp/yamnet/query_embedding/query/fast/20.h5
[del] will it be true:  True
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/36.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/fast/36.h5 False
[del] if statement TRUE! reading  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/36.ogg
[del] wav_data:  [[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
[del] scores:  [[9.9998975e-01 2.2464677e-14 2.3109433e-06 ... 8.4159841e-26
  5.9418502e-15 0.0000000e+00]
 [9.9906492e-01 1.8829407e-05 3.3134818e-03 ... 1.3296752e-08
  8.3930099e-06 5.4765150e-17]
 [9.9940741e-01 6.3858119e-05 4.695415

100%|██████████| 66/66 [00:08<00:00,  7.74it/s]

[del] will it be true:  True
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/22.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/fast/22.h5 False
[del] if statement TRUE! reading  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/22.ogg
[del] wav_data:  [[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
[del] scores:  [[9.9916768e-01 2.9353109e-10 4.6280792e-07 ... 2.4595350e-20
  1.9008827e-12 4.2877766e-38]
 [9.9980807e-01 3.5422028e-07 2.4470687e-04 ... 1.4847207e-09
  4.1990811e-06 2.3493590e-20]
 [9.9910122e-01 1.3392297e-05 5.9873462e-03 ... 1.5422312e-06
  3.6141276e-04 3.6549695e-14]
 [9.9508935e-01 2.4452806e-04 8.0535412e-03 ... 8.7731089e-05
  5.2979589e-04 1.7512738e-10]]
[del] Will also save the embedding, to:  ../data/sp/yamnet/query_embedding/query/fast/22.h5
[del] will it be true:  False
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/fast/23.ogg True
Should not exist:  ..

In [30]:

test_input_path = 'test/query/fast/8.ogg'
test2 = os.path.abspath(test_input_path)
print(test2)

# test2 = '../8.ogg'

# test2 = '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/slow/8.ogg'
wav_data, sr = sf.read(test2, dtype=np.int16)



# /Users/casper/Documents/UvA\ master/b23456_thesis/msc_thesis/code/data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/slow/8.ogg




/Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/scripts/test/query/fast/8.ogg
